In [1]:
import tensorflow as tf
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from scipy.stats import norm
from sklearn.preprocessing import StandardScaler
from scipy import stats
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline


np.random.seed(0)

In [2]:
df_train = pd.read_csv('train.csv')
df_train.head(5)

,label,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [3]:
[len(df_train[df_train['label'] == i])/len(df_train) for i in range(10)]

[0.09838095238095237,
 0.11152380952380953,
 0.09945238095238096,
 0.1035952380952381,
 0.09695238095238096,
 0.09035714285714286,
 0.0985,
 0.10478571428571429,
 0.09673809523809523,
 0.09971428571428571]

Можно говорить о том, что каждый класс представлен примерно тем же количеством предсавителей, что и остальные классы.
Поэтому можно не использовать стратификацию.

In [4]:
labels = df_train['label']
df_train.drop('label', axis=1, inplace=True)

In [5]:
df_train = df_train.astype(np.float)

# convert from [0:255] => [0.0:1.0]
df_train = np.multiply(df_train, 1.0 / 255.0)

In [6]:
from sklearn.model_selection import train_test_split


X_train, X_test, y_train, y_test = train_test_split(df_train, labels, test_size=0.20, random_state=42)

In [12]:
def dense_to_one_hot(labels_dense, num_classes):
    num_labels = labels_dense.shape[0]
    index_offset = np.arange(num_labels) * num_classes
    labels_one_hot = np.zeros((num_labels, num_classes))
    labels_one_hot.flat[index_offset + labels_dense.ravel()] = 1
    return labels_one_hot

In [14]:
y_train_one_hot = dense_to_one_hot(y_train, 10).astype(np.uint8)
y_test_one_hot = dense_to_one_hot(y_test, 10).astype(np.uint8)

## Model 1

Для начала попробуем построить простую мультиклассовую логистическую регрессию. 

Точность полученную с помощью данной модели мы будем считать за пороговое значение при сравнении с другими более сложными моделями.

### Inference

Итак, наша модель есть $$l = x \cdot W + b$$ $$ \hat{y} = \text{softmax}(l) $$

Ошибка: кросс-энтропия $$ \mathcal{L} = \sum\limits_{i=0}^{N} y_i \log \hat{y}_i $$

In [7]:
from sklearn.metrics import accuracy_score


X = tf.placeholder(tf.float32, [None, 784])
Y = tf.placeholder(tf.float32, [None, 10])

w = tf.Variable(tf.random_normal(shape=[784, 10], stddev=0.01), name="weights")
b = tf.Variable(tf.zeros([10]), name="bias")

logits = tf.matmul(X, w) + b

entropy = tf.nn.softmax_cross_entropy_with_logits(logits, Y)
loss = tf.reduce_mean(entropy) 

### Training

Обучаем модель методом градиентного спуска.

In [19]:
optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.01).minimize(loss)

n_epochs = 100
init = tf.initialize_all_variables()

sess = tf.Session()
sess.run(init)

for i in range(n_epochs): 
    for x,y in zip(X_train.values, y_train_one_hot):
        sess.run([optimizer], feed_dict={X: [x], Y:[y]})

### Evaluation

Метрикой качества в данном случае является ```accuracy``` - процент верно предсказанных меток.

In [20]:
correct_prediction = tf.equal(tf.argmax(logits,1), tf.argmax(Y,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

print(sess.run(accuracy, feed_dict={X: X_test.values, Y: y_test_one_hot}))

0.909643


In [ ]:
sess.close()